<a href="https://colab.research.google.com/github/i-murray/SIGNATEHiroshimaLemon/blob/main/CreateArchieve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upgrage / Install
!pip install pandas
!pip install h5py
!pip install opencv-python
!pip install numpy

In [1]:
import pandas as pd
import h5py
import cv2
import numpy as np
from collections import defaultdict
import os

from shutil import copyfile
copyfile('drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/train_images.zip','train_images.zip')
copyfile('drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/test_images.zip','test_images.zip')
import zipfile
with zipfile.ZipFile('train_images.zip', 'r') as zip_ref:
    zip_ref.extractall()
with zipfile.ZipFile('test_images.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [2]:
with h5py.File('archive.hdf5', 'w') as hf:

  for split in ['train','test']:
    if split == 'train':
      images = defaultdict(list)
    else:
      images = []

    for _, row in pd.read_csv(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/{split}_images.csv').iterrows():
      #if os.path.exists(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/{split}_images/'+row['id']):
      if os.path.exists(f'{split}_images/'+row['id']):
        #im = cv2.imread(f'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/{split}_images/'+row['id'])
        im = cv2.imread(f'{split}_images/'+row['id'])
        im = im[:, :, ::-1]
        if split == 'train':
          images[row['class_num']].append(im)
        else:
          images.append(im)

    if split == 'train':
      grp = hf.create_group(split)
      for k in images.keys():
        d = np.stack(images[k],axis=0)
        grp.create_dataset(str(k),
                        data=d,
                        shape=d.shape,
                        maxshape=d.shape,
                        compression="gzip",
                        compression_opts=9,
                        chunks=True,
                        dtype=np.uint8)
    else:
      d = np.stack(images,axis=0)
      hf.create_dataset(split,
                      data=d,
                      shape=d.shape,
                      maxshape=d.shape,
                      compression="gzip",
                      compression_opts=9,
                      chunks=True,
                      dtype=np.uint8)  
  hf.close()

In [4]:
copyfile('archive.hdf5','drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/archive.hdf5')

'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/archive.hdf5'